In [ ]:
"""%pip install requests
   %pip install pandas"""

In [5]:
import requests
import pandas as pd

# Define the list of currencies you need
CURRENCIES = ['EUR', 'GBP', 'HKD', 'HUF', 'ILS', 'INR', 'JPY', 'KES', 'KRW', 'KWD', 'MXN', 'NOK', 'NZD', 'PLN', 'QAR', 'RON', 'RUB', 'SEK', 'SGD', 'THB', 'TRY', 'TWD', 'USD']

def fetch_frankfurter_rates():
    """Fetch exchange rates from Frankfurter.app."""
    url = "https://api.frankfurter.app/latest?from=USD"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.json()['rates']
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Frankfurter.app: {e}")
        return {}

def fetch_fallback_rates():
    """Fetch exchange rates from a fallback API (exchangerate-api.com)."""
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()['rates']
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from fallback API: {e}")
        return {}

def get_exchange_rates():
    """Combine rates from Frankfurter.app and a fallback API."""
    # Fetch rates from Frankfurter.app
    frankfurter_rates = fetch_frankfurter_rates()
    
    # Fetch rates from the fallback API
    fallback_rates = fetch_fallback_rates()
    
    # Combine rates, prioritizing Frankfurter.app data
    rates = {}
    for currency in CURRENCIES:
        if currency in frankfurter_rates:
            rates[currency] = frankfurter_rates[currency]
        elif currency in fallback_rates:
            rates[currency] = fallback_rates[currency]
        else:
            print(f"Rate for {currency} not found in any API.")
            rates[currency] = None
    
    # Create a DataFrame
    df = pd.DataFrame([
        {'RatCurrencyFrom': currency, 'RatCurrencyTo': 'USD', 'RatAmount': 1 / rate if rate else None, 'RatType': 'COBU'}
        for currency, rate in rates.items()
    ])
    return df

if __name__ == "__main__":
    df = get_exchange_rates()
    if not df.empty:
        print(df)
        # Save the DataFrame to a CSV file
        df.to_csv('exchange_rates.csv', index=False)
    else:
        print("Failed to fetch exchange rates.")

   RatCurrencyFrom RatCurrencyTo  RatAmount RatType
0              EUR           USD   1.083001    COBU
1              GBP           USD   1.292708    COBU
2              HKD           USD   0.128660    COBU
3              HUF           USD   0.002706    COBU
4              ILS           USD   0.273336    COBU
5              INR           USD   0.011488    COBU
6              JPY           USD   0.006742    COBU
7              KES           USD   0.007738    COBU
8              KRW           USD   0.000686    COBU
9              KWD           USD   3.244373    COBU
10             MXN           USD   0.049606    COBU
11             NOK           USD   0.093382    COBU
12             NZD           USD   0.568666    COBU
13             PLN           USD   0.258071    COBU
14             QAR           USD   0.274725    COBU
15             RON           USD   0.217581    COBU
16             RUB           USD   0.011514    COBU
17             SEK           USD   0.097832    COBU
18          